importing all the statements

In [ ]:
import sys
import os

parent_folder = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_folder)


In [1]:
# imports 
import torch 
import math
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from datasets import load_from_disk
from transformers import AutoTokenizer
from Training_utils.buildModel import BuildModel
from Training_utils.training_tools import Train_tools
from functools import partial
from torch.amp import GradScaler

/home/gz/Documents/Full Pipeline(LLM)/.llm_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPTModel class defined.


Tokenizer and Data Loading

In [ ]:
# loading tokenizer 

tokenizer_path = '../Saved_tokenizer/t5_Tokinzer'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path,use_fast=True)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:

# preproccesd data load
path = "../Saved_Data/Tiny_chat_pretrained_dataset"
dataset = load_from_disk(path)

In [5]:
dataset = dataset.remove_columns(['raw_text'])

In [6]:
print(dataset)

Dataset({
    features: ['input_ids'],
    num_rows: 1000000
})


In [ ]:
lengths = [len(row['input_ids']) for row in dataset]

# Compute stats
avg_len = sum(lengths) / len(lengths)
min_len = min(lengths)
max_len = max(lengths)

print(f"Number of sequences: {len(dataset)}")
print(f"Average tokens per sequence: {avg_len:.1f}")
print(f"Minimum tokens: {min_len}")
print(f"Maximum tokens: {max_len}")

 helper variables def

In [3]:
vocab_size = len(tokenizer)
ignore_index = tokenizer.pad_token_type_id
device = ('cuda' if torch.cuda.is_available() else 'cpu')
max_len = 200

Dataloader and data spliting


In [27]:
#collate fn 

def collate_fn(batch,pad_token=ignore_index,device=device,max_length=max_len):
   
    
   

    inputs_list = []
    target_list = []
    truncated_seqs = []
    for item in batch:
        inputIds = item['input_ids']
        if max_length is not None and len(inputIds) > max_length:
            inputIds = inputIds[:max_length-1] + [inputIds[-1]]
        truncated_seqs.append(inputIds)

    batch_max_len = max(len(seq) + 1 for seq in truncated_seqs)
    
    for inputIds in truncated_seqs:

        new_input = list(inputIds) + [pad_token]
       
        padded = (
            new_input + [pad_token] * (batch_max_len - len(new_input)))
    
        inputs_list.append(torch.tensor(padded[:-1],dtype=torch.long,device=device))
        target_list.append(torch.tensor(padded[1:],dtype=torch.long,device=device))

        
    input_ids = torch.stack(inputs_list)
    labels = torch.stack(target_list)


    return (input_ids,
            labels)
   

collate_fn_pre_loaded = partial(collate_fn,pad_token=ignore_index,device=device,max_length=max_len)

In [28]:
train_percentage = int(len(dataset) * 0.98)

train_data = dataset.select(range(train_percentage))
val_data = dataset.select(range(len(train_data),len(dataset)))
print(f'training samples: {len(train_data)}')
print(f'validation samples: {len(val_data)}')


training samples: 980000
validation samples: 20000


In [31]:
# DataLoaders
Train_loader = DataLoader(train_data,batch_size=16,collate_fn=collate_fn_pre_loaded,shuffle=True,drop_last=False)
Val_Loader = DataLoader(val_data,batch_size=16,collate_fn=collate_fn_pre_loaded,shuffle=False,drop_last=False)

In [32]:
# num of batches
num_batches = len(Train_loader)
print(num_batches)

61250


In [4]:
model_config = {
    'D_Model': 420,
    'Num_Heads': 6,
    'Num_Layers':6,
    'Dropout': 0.05,    
    'Vocab_size': vocab_size,
    'FeedForward_size': 2000,
    'Context_size':200

}


'''train_config = {
        'learning_rate': 0.001, 

        'GradAccumulation_steps': 1,
        'Weight_decay': 0.05,
        'Epochs': 1,
        'Label_smoothing': 0.05,
        'warmupsteps_percentage':0.30,
        'Num_batches': num_batches,
        'ignore_index': ignore_index,
    

} '''



"train_config = {\n        'learning_rate': 0.001, \n\n        'GradAccumulation_steps': 1,\n        'Weight_decay': 0.05,\n        'Epochs': 1,\n        'Label_smoothing': 0.05,\n        'warmupsteps_percentage':0.30,\n        'Num_batches': num_batches,\n        'ignore_index': ignore_index,\n\n\n} "

In [5]:
# crating the model 

builder = BuildModel(model_type='GModel2')
model = builder.createModel(config=model_config,Model_type='gmodel2')
model.to(device)
model.load_state_dict(torch.load('/home/gz/Documents/Full Pipeline(LLM)/Saved_Models/Tiny_Chat_41m_Pretrained/41m200T_pretrained_Tchat.pth'))
# Training tools
#loss_fn,optimzer,lr_scheduler = Train_tools(config=train_config).getTools(model.parameters())


/tmp/ipykernel_10272/1605091033.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/home/gz/Documents/Full Pipeline(LLM)/Saved_Models/Tiny

<All keys matched successfully>

In [41]:
print(builder.get_total_params(in_millons=True))

41.343965


In [43]:
import os
CheckPoint_Path = '../Full Pipeline(LLM)/Saved_Models/Tiny_Chat_PreTrained/CheckPoints'
os.path.exists(CheckPoint_Path)

True

In [ ]:
def train(model, optimizer, lr_scheduler, loss_fn, Train_Loader, Val_Loader, Epochs, grad_step, pad_token, device, CheckPoint_Path=CheckPoint_Path):
    #scaler = GradScaler()
    for epoch in range(Epochs):
        progress_bar = tqdm(Train_Loader, desc=f"Epoch {epoch+1} training", leave=False)
        total_loss = 0.0
        model.train()
        for step, (input_ids, labels) in enumerate(progress_bar):
            with torch.autocast(device_type=device, dtype=torch.bfloat16): 
                logits = model(input_ids, pad_token)
                loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
            total_loss += loss.item()
            loss = loss / grad_step
            loss.backward()  
            if (step + 1) % grad_step == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                min_lr = 5e-5
                for g in optimizer.param_groups:
                    g['lr'] = max(g['lr'], min_lr)
                optimizer.zero_grad()
            progress_bar.set_postfix({'loss': total_loss / (step + 1)})

        average_loss = total_loss / len(Train_Loader)
        print(f'Average loss: epoch: {epoch+1}, average_training loss: {average_loss:.4f}')
        torch.save(
            {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': average_loss,
                
            },
            f"{CheckPoint_Path}/checkpoint{epoch+1}.pth"
        )

        val_loss = 0.0
        model.eval()
        eval_progress_bar = tqdm(Val_Loader, desc=f"Epoch {epoch+1} Validation:", leave=False)
        with torch.no_grad():
            for input_ids, labels in eval_progress_bar:
                with torch.autocast(device_type=device, dtype=torch.bfloat16):  # <-- change here
                    logits = model(input_ids, pad_token)
                    loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))

                val_loss += loss.item()
                eval_progress_bar.set_postfix({'loss': val_loss / max(eval_progress_bar.n, 1)})
        average_eval_loss = val_loss / len(Val_Loader)
        perplexity = math.exp(average_eval_loss)
        print(f'Average Evaluation Loss: {epoch+1} {average_eval_loss:.4f}')
        print(f'Perplexity Score: {perplexity:.2f}')
        current_lr = optimizer.param_groups[0]['lr']
        print(f'Current Learning Rate: {current_lr:.8f}')


In [45]:


train(model,optimzer,lr_scheduler,loss_fn,Epochs=train_config['Epochs'],Train_Loader=Train_loader,Val_Loader=Val_Loader,grad_step=train_config['GradAccumulation_steps'],pad_token=ignore_index,device=device,CheckPoint_Path=CheckPoint_Path)



Average loss: epoch: 1, average_training loss: 2.3904


Average Evaluation Loss: 1 2.0417
Perplexity Score: 7.70
Current Learning Rate: 0.00005000


In [55]:
from pathlib import Path
import json
def Save_model(model_name,folder_name,model):
        
        save_folder = Path(f"../Full Pipeline(LLM)/Saved_Models/{folder_name}")
        save_folder.mkdir(parents=True, exist_ok=True)

        torch.save(model.state_dict(),save_folder/f'{model_name}.pth')

        print(f'model has been saved in folder {save_folder}')
        print(f'Model weights saved at: {save_folder}/{model_name}.pth')
def save_config(folder_name,file_name,model_config,train_config):


        config_path = f"../Full Pipeline(LLM)/Saved_Models/{folder_name}/{file_name}.json"

        all_configs = {
                "model_config": model_config,
                "train_config": train_config
        }

        with open(config_path, 'w') as f:
                json.dump(all_configs, f, indent=4)

In [56]:
# saving the model
Save_model(folder_name='Tiny_Chat_41m_Pretrained',model_name='41m200T_pretrained_Tchat',model=model)
#builder.save_config(folder_name='PreTrain_1',model_config=model_config,train_config=train_config)

model has been saved in folder ../Full Pipeline(LLM)/Saved_Models/Tiny_Chat_41m_Pretrained
Model weights saved at: ../Full Pipeline(LLM)/Saved_Models/Tiny_Chat_41m_Pretrained/41m200T_pretrained_Tchat.pth


In [57]:
save_config(folder_name='Tiny_Chat_41m_Pretrained',file_name='Pretrained_Tchat_41m_config',model_config=model_config,train_config=train_config)

In [6]:
def gen_text(model, tokenizer, prompt, max_tokens=100, pad_token=0, device=None):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # Build the formatted prompt
    prompt = f"{tokenizer.bos_token} <user> {prompt} <bot> "
    
    # Encode prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt',add_special_tokens=False).to(device)

    model.eval()
    generated_ids = input_ids

    for _ in range(max_tokens):
        if generated_ids.size(1) >= 285:   # Your sequence limit
            break

        # Forward pass — only input_ids and pad_token needed
        with torch.no_grad():
            logits = model(generated_ids, pad_token=pad_token)

        # Take last token's logits
        next_logits = logits[:, -1, :]

        # Greedy decode
        next_id = torch.argmax(next_logits, dim=-1).unsqueeze(0)

        # Stop on custom end token
        if tokenizer.decode(next_id[0]) == "<end>" or tokenizer.decode(next_id[0]) == "<user>" :
            break

        # Append new token
        generated_ids = torch.cat([generated_ids, next_id], dim=-1)

    # Decode into text
    text = tokenizer.decode(generated_ids[0].tolist())

    # Extract only bot response
    words = text.split()
    if "<bot>" in words:
        bot_idx = words.index("<bot>")
        words = words[bot_idx:]
    return " ".join(words)


In [12]:
text = " why do people fall in love so fast ? " 

print(gen_text(model,tokenizer,prompt=text,max_tokens=200))

<start><user> why do people fall in love so fast?<bot> it can feel very unfair when others do not show respect for others.


In [66]:
import re
def clean_text(text):
    text = text.lower()

    tokens = re.findall(r"\w+(?:'\w+)*|[^\w\s]", text)
    
    tokens = " ".join(tokens)
    return tokens

prompts = [
    "hey how are you?",
    "what is the meaning of love ?",
    "do you like watching movies ?",
    "do you like reading books or watching movies better?",
    " Tell me the best way to learn to read.",
    "I'm doing well, how about you?."
    
]
responses = [ ]
for prompt in prompts:
    
    prompt = clean_text(prompt)
   
    
    output = gen_text(model, tokenizer, prompt,max_tokens=200)
    responses.append(output)
    print(f"input: {prompt}")
    print(f"output: {output}")
    print("______________")
 
  


input: hey how are you ?
output: <start><user> hey how are you?<bot> i am feeling very happy today, thank you for asking.
______________
input: what is the meaning of love ?
output: <start><user> what is the meaning of love?<bot> love is a strong feeling that brings people together in many ways.
______________
input: do you like watching movies ?
output: <start><user> do you like watching movies?<bot> i enjoy watching movies that have a wide range of interesting characters and plots.
______________
input: do you like reading books or watching movies better ?
output: <start><user> do you like reading books or watching movies better?<bot> i enjoy reading books because they take me to different worlds.
______________
input: tell me the best way to learn to read .
output: <start><user> tell me the best way to learn to read.<bot> that sounds interesting and surprising. what kind of book do you enjoy reading?
______________
input: i'm doing well , how about you ? .
output: <start><user> i'm 

In [65]:
model.transformer_blocks

ModuleList(
  (0-5): 6 x TransformerBlock(
    (Attention): MultiHeadAttention(
      (WQ): Linear(in_features=420, out_features=420, bias=True)
      (WK): Linear(in_features=420, out_features=420, bias=True)
      (WV): Linear(in_features=420, out_features=420, bias=True)
      (Final_layer): Linear(in_features=420, out_features=420, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (FeedForward): PointwiseFeedForward(
      (fc1): Linear(in_features=420, out_features=2000, bias=True)
      (fc2): Linear(in_features=2000, out_features=420, bias=True)
      (gelu): GELU(approximate='none')
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (RMSNorm1): RMSNorm()
    (RMSNorm2): RMSNorm()
    (dropout): Dropout(p=0.05, inplace=False)
  )
)